In [1]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
"""
TODO:
- MSEloss for all layer tests
- Proper data generation method (multiple datasets needed to test everything)
- Test for ints and big float as well

"""

In [2]:
# Taken from I2DL exercises, need to cite!

# def eval_numerical_gradient_array(f, x, df, h=1e-5):
#     """
#     Evaluate a numeric gradient for a function that accepts a numpy
#     array and returns a numpy array.
#     """
#     grad = np.zeros_like(x)
#     it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
#     while not it.finished:
#         ix = it.multi_index

#         oldval = x[ix]
#         x[ix] = oldval + h
#         pos = f(x).copy()
#         x[ix] = oldval - h
#         neg = f(x).copy()
#         x[ix] = oldval

#         grad[ix] = np.sum((pos - neg) * df) / (2 * h)
#         it.iternext()
#     return grad

In [51]:
# ReLU testdata generation

# First val is batch size, rest are dimensions of data.
# Numbers are absed on input shape of MTT ReLU layers
# shape = (256, 128, 32, 32)
shape = (4, 3, 2, 2)

x = torch.randn(*shape, dtype=torch.float, requires_grad=True)
relu = nn.ReLU()
out = relu(x)  # Forward pass

target = torch.randn(*shape)  # Imaginary target values, defines shape of dout
loss_fn = nn.MSELoss()
loss = loss_fn(out, target)  # Compute loss
loss.backward()  # Backward pass

# Validation
dout = 2 * (out - target) / np.prod(shape) # derivative of MSELoss
cache = x.clone().detach() # Must clone x so it isn't overwritten
cache[cache <= 0] = 0
dx = cache
dx[dx > 0] = 1
dx_truth = dout * dx
print(f"x gradients are approximately the same: {torch.allclose(dx_truth, x.grad)}")

# File name: shape_variable_testNum
with open(f'relu_{shape[0]}_{shape[1]}_{shape[2]}_{shape[3]}_x_t1.npy', 'wb') as f:
    np.save(f, x.detach().numpy())
with open(f'relu_{shape[0]}_{shape[1]}_{shape[2]}_{shape[3]}_dx_t1.npy', 'wb') as f:
    np.save(f, x.grad.detach().numpy())
with open(f'relu_{shape[0]}_{shape[1]}_{shape[2]}_{shape[3]}_out_t1.npy', 'wb') as f:
    np.save(f, out.detach().numpy())
with open(f'relu_{shape[0]}_{shape[1]}_{shape[2]}_{shape[3]}_dout_t1.npy', 'wb') as f:
    np.save(f, dout.detach().numpy())


x gradients are approximately the same: True


In [52]:
# print(x.grad)
# print(cache)
# print(dout)

tensor([[[[ 0.0000,  0.0000],
          [-0.0088,  0.0525]],

         [[ 0.0000,  0.0000],
          [ 0.0750,  0.0000]],

         [[ 0.0094,  0.0726],
          [ 0.1179,  0.0531]]],


        [[[ 0.0516,  0.0000],
          [ 0.0000,  0.0424]],

         [[ 0.1115,  0.0000],
          [ 0.1554,  0.0000]],

         [[ 0.0000,  0.0000],
          [ 0.0397,  0.1478]]],


        [[[ 0.0000, -0.0853],
          [ 0.0153,  0.0000]],

         [[ 0.0000,  0.0000],
          [ 0.0388,  0.0000]],

         [[ 0.0000,  0.0000],
          [ 0.0000,  0.1355]]],


        [[[ 0.0000,  0.0079],
          [-0.0631,  0.0778]],

         [[ 0.0000,  0.0000],
          [ 0.0000,  0.0038]],

         [[ 0.0000,  0.0000],
          [ 0.0000,  0.0000]]]])


In [52]:
# Affine/Linear layer testdata generation (using arange to create specific weights)

in_shape = (3, 2, 2) # First val is batch size, rest are input dimensions
layer_shape = (np.prod(in_shape[1:]), 14) # Shape of linear layer, in_features and out_features

x = torch.arange(np.prod(in_shape), dtype=torch.float, requires_grad=True)
x = x.reshape((in_shape[0], np.prod(in_shape[1:]))) # Reshape to shape [batch, prod(input_shape)]
x.retain_grad() # Needed to calculate gradients for tensor views, i.e. reshape

linear = nn.Linear(*layer_shape)
w = reversed(torch.arange(np.prod(linear.weight.shape), dtype=torch.float))
w = w.reshape(linear.weight.shape)
linear.weight = torch.nn.Parameter(w)
b = torch.full(linear.bias.shape, 2, dtype=torch.float)
linear.bias = torch.nn.Parameter(b)
out = linear(x) # Forward pass

target = torch.randn(in_shape[0], layer_shape[1])  # Imaginary target values, defines shape of dout
loss_fn = nn.MSELoss()
loss = loss_fn(out, target)  # Compute loss
loss.backward()

# Validation
dout = 2 * (out - target) / np.prod((in_shape[0], layer_shape[1])) # derivative of MSELoss
dx_truth = dout @ w
dw_truth = dout.T @ x
db_truth = torch.sum(dout.T, axis=1)
print(f"x gradients are approximately the same: {torch.allclose(dx_truth, x.grad)}")
print(f"w gradients are approximately the same: {torch.allclose(dw_truth, linear.weight.grad)}")
print(f"b gradients are approximately the same: {torch.allclose(db_truth, linear.bias.grad)}")


x gradients are approximately the same: True
w gradients are approximately the same: True
b gradients are approximately the same: True


In [27]:
# Affine/Linear layer testdata generation (random tensors)
# Remember to make dimensions divisible by block shape (usually 2^n)

in_shape = (8, 3, 2, 2) # First val is batch size, rest are input dimensions
layer_shape = (np.prod(in_shape[1:]), 4) # Shape of linear layer, in_features and out_features

x = torch.randn(*in_shape, requires_grad=True)
x = x.reshape((in_shape[0], np.prod(in_shape[1:]))) # Reshape to shape [batch, prod(input_shape)]
x.retain_grad() # Needed to calculate gradients for tensor views, i.e. reshape

linear = nn.Linear(*layer_shape)
out = linear(x) # This output will also be our dout, since loss is just a sum

target = torch.randn(in_shape[0], layer_shape[1])  # Imaginary target values, defines shape of dout
loss_fn = nn.MSELoss()
loss = loss_fn(out, target)  # Compute loss
loss.backward()

# Validation
dout = 2 * (out - target) / np.prod((in_shape[0], layer_shape[1])) # derivative of MSELoss
dx_truth = dout @ linear.weight
dw_truth = dout.T @ x
db_truth = torch.sum(dout, axis=0)
out_truth = x @ linear.weight.T + linear.bias
print(f"x gradients are approximately the same: {torch.allclose(dx_truth, x.grad)}")
print(f"w gradients are approximately the same: {torch.allclose(dw_truth, linear.weight.grad)}")
print(f"b gradients are approximately the same: {torch.allclose(db_truth, linear.bias.grad)}")
print(f"out values are approximately the same: {torch.allclose(out_truth, out)}")

it = 1
# File name: shape_variable_testNum
name = f'in_{in_shape[0]}_{in_shape[1]}_{in_shape[2]}_{in_shape[3]}_out_{layer_shape[1]}'
with open(f'linear_{name}_x_t{it}.npy', 'wb') as f:
    np.save(f, x.detach().numpy())
with open(f'linear_{name}_w_t{it}.npy', 'wb') as f:
    np.save(f, linear.weight.detach().numpy())
with open(f'linear_{name}_b_t{it}.npy', 'wb') as f:
    np.save(f, linear.bias.detach().numpy())
with open(f'linear_{name}_out_t{it}.npy', 'wb') as f:
    np.save(f, out.detach().numpy())
with open(f'linear_{name}_dx_t{it}.npy', 'wb') as f:
    np.save(f, x.grad.detach().numpy())
with open(f'linear_{name}_dw_t{it}.npy', 'wb') as f:
    np.save(f, linear.weight.grad.detach().numpy())
with open(f'linear_{name}_db_t{it}.npy', 'wb') as f:
    np.save(f, linear.bias.grad.detach().numpy())
with open(f'linear_{name}_dout_t{it}.npy', 'wb') as f:
    np.save(f, dout.detach().numpy()) # Note this is what we computed manually

x gradients are approximately the same: True
w gradients are approximately the same: True
b gradients are approximately the same: True
out values are approximately the same: True


In [40]:
# print(linear.weight.shape)
# print(linear.weight.grad.shape)
# print(linear.weight.grad.T)
# print(linear.weight.shape)
# print(linear.bias.shape)

# print(linear.weight)
# print(linear.weight.T)
# print(x)
# print(x @ linear.weight.T)
# print(out.shape)
# print(dout.shape)
# print(dout)
# print(out_truth)

# print(dout)
# print(torch.sum(dout, axis=0))
# print(torch.sum(dout, axis=0).shape)
# print(linear.bias.shape)

print(dout.T.shape)
print(dout.T)
print(linear.weight.grad)
print(linear.bias.grad)

# test = torch.randn(*in_shape)
# print(test.shape)
# print(test.T.shape)

# print(test[0][0][1][1])
# print(test.T[1][1][0][0])


torch.Size([4, 8])
tensor([[ 0.0060,  0.0087, -0.1293,  0.1165, -0.0031,  0.0360, -0.1315,  0.0180],
        [ 0.1446,  0.0189, -0.0035,  0.0265, -0.1367,  0.0506,  0.0139,  0.0590],
        [-0.1067, -0.0135,  0.0700,  0.0213,  0.0282,  0.1001, -0.0951, -0.0397],
        [-0.0617, -0.0492,  0.0075,  0.0293, -0.0469, -0.0548,  0.0808, -0.0176]],
       grad_fn=<PermuteBackward0>)
tensor([[-0.0719, -0.4019,  0.1952, -0.4095,  0.1383, -0.0644, -0.0371, -0.1648,
          0.0560, -0.1487, -0.0990,  0.0757],
        [-0.5282,  0.2830, -0.3337, -0.2315, -0.0621, -0.1323,  0.1940, -0.5524,
          0.0017, -0.0242, -0.2560,  0.0116],
        [-0.0467, -0.4511,  0.3410, -0.0869, -0.1210,  0.3099,  0.0410,  0.1691,
          0.0357, -0.1220,  0.2101, -0.0284],
        [ 0.0727,  0.0104, -0.1876,  0.3582,  0.0361, -0.1230, -0.1675,  0.0945,
         -0.1885,  0.1646,  0.0105, -0.0225]])
tensor([-0.0786,  0.1733, -0.0354, -0.1126])
